In [ ]:
# Import required libraries and dependencies
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame and make the index the "coin_id" column.
market_data_df = pd.read_csv("Resources/crypto_market_data.csv", index_col="coin_id")

# Display sample data
market_data_df.head(10)

In [ ]:
# Generate summary statistics
market_data_df.describe()

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file by scaling the features of the DataFrame.
crypto_scaled = StandardScaler().fit_transform(market_data_df)


In [ ]:
# Create a DataFrame with the scaled data
crypto_scaled_df = pd.DataFrame(
    crypto_scaled, columns=market_data_df.columns
)

# Copy the crypto names from the original data
crypto_scaled_df["coin_id"] = market_data_df.index

# Set the coinid column as index
crypto_scaled_df = crypto_scaled_df.set_index("coin_id")

# Display sample data
crypto_scaled_df.head()



### Find the Best Value for k Using the Original Scaled DataFrame.

In [ ]:
# Create a list with the number of k-values to try 
# Use a range from 1 to 11
k = list(range(1, 11))

# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using the scaled DataFrame
# 3. Append the model.inertia_ to the inertia list
for i in k:
    kmodel = KMeans(n_clusters=i,n_init="auto")
    kmodel.fit(crypto_scaled_df)
    inertia.append(kmodel.inertia_)

# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}
# Display sample data
elbow_data

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

# Display the DataFrame
elbow_df.head(10)


In [ ]:
# Plot a line chart with all the inertia values computed with the different values of k

# Set the x-axis to "k"
# Set the y-axis to "inertia"
# Set the x and y axis titles

elbow_df.plot(x="k", y="inertia", title="Elbow Curve")  


#### Answer the following question: 
**Question:** What is the best value for `k`?

**Answer:**
The best value for k is 4 for our data. I determined this because the line becomes relatively 'flat' after that point. Therefore the optimal number of clusters for the data is 4.

### Cluster Cryptocurrencies with K-means Using the Original Scaled Data.

In [ ]:
# Initialize the K-Means model using the best value for k
# Use 4 as the k value
model = KMeans(n_clusters=4)


In [ ]:
# Fit the K-Means model using the scaled data
model.fit(crypto_scaled_df)


In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters = model.predict(crypto_scaled_df)

# View the resulting array of cluster values.
print(crypto_clusters)


In [ ]:
# Create a copy of the DataFrame
clustered_df = crypto_scaled_df.copy()


In [ ]:
# Add a new column to the DataFrame with the predicted clusters
clustered_df["CoinName"] = market_data_df["CoinName"]

# Display sample data
clustered_df.head()


In [ ]:
# Create a scatter plot using Pandas plot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Use "rainbow" for the color to better visualize the data.
clustered_df.plot.scatter(x="price_change_percentage_24h", y="price_change_percentage_7d", c="Class", colormap="rainbow", title="Cryptocurrencies Clusters")


### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.


In [ ]:
# Use the PCA model with `fit_transform` on the original scaled DataFrame to reduce to three principal components.


# View the first five rows of the DataFrame. 


In [ ]:
# Retrieve the explained variance to determine how much information  can be attributed to each principal component.


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [ ]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data


# Copy the crypto names from the original data


# Set the coinid column as index


# Display sample data


### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11


# Create an empty list to store the inertia values


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using PCA DataFrame.
# 3. Append the model.inertia_ to the inertia list


# Create a dictionary with the data to plot the Elbow curve


# Create a DataFrame with the data to plot the Elbow curve


# Display the DataFrame


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.


#### Answer the following questions: 
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k


In [ ]:
# Fit the K-Means model using the PCA data


In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data


# Print the resulting array of cluster values.


In [ ]:
# Create a copy of the DataFrame with the PCA data


# Add a new column to the DataFrame with the predicted clusters


# Display sample data


In [ ]:
# Create a scatter plot using hvPlot by setting `x="PCA1"` and `y="PCA2"`. 


### Determine the Weights of Each Feature on each Principal Component

In [ ]:
# Use the columns from the original scaled DataFrame as the index.


#### Answer the following question: 

* **Question:** Which features have the strongest positive or negative influence on each component? 
 
* **Answer:** 
    